In [3]:
!pip install -Uq upgini catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.3/108.3 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 913.9/913.9 kB 19.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.7/162.7 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.0/447.0 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from upgini import FeaturesEnricher, SearchKey
from upgini.metadata import CVType
from catboost import CatBoostRegressor
from catboost.utils import eval_metric

In [5]:
df = pd.read_csv('/content/train.csv')

In [6]:
df.shape

(227618, 4)

In [24]:
df = df.sample(n=12_000, random_state=1)

In [25]:
df.shape

(12000, 5)

In [26]:
df.head()

,index,date,store,item,sales
7576,212737,2015-07-11,7,12,82.0
10509,119967,2016-07-01,6,7,67.0
4253,221466,2014-06-05,2,13,143.0
5150,112018,2014-09-25,2,7,62.0
506,27456,2013-03-08,6,2,43.0


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12000 entries, 7576 to 6060
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   index   12000 non-null  int64         
 1   date    12000 non-null  datetime64[ns]
 2   store   12000 non-null  int64         
 3   item    12000 non-null  int64         
 4   sales   12000 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 562.5 KB


In [28]:
# Converting the data object into pandas' date field

df['date'] = pd.to_datetime(df['date'])

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12000 entries, 7576 to 6060
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   index   12000 non-null  int64         
 1   date    12000 non-null  datetime64[ns]
 2   store   12000 non-null  int64         
 3   item    12000 non-null  int64         
 4   sales   12000 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 562.5 KB


In [30]:
df.sort_values('date', inplace=True)

In [31]:
# Removing the random index
df.reset_index(inplace=True)

In [32]:
df.head()

,level_0,index,date,store,item,sales
0,6,222772,2013-01-01,3,13,49.0
1,3,113212,2013-01-01,3,7,40.0
2,7,51128,2013-01-01,9,3,20.0
3,4,40172,2013-01-01,3,3,26.0
4,8,142428,2013-01-01,9,8,45.0


Splitting Our Data

In [33]:
X = df.drop(columns="sales")
y = df['sales']

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

**Enriching Our Features**

Having only 3 features to predict a target value is not ideal. Hence, we will be using *upgini* to create new relevant features.

In [35]:
enricher = FeaturesEnricher(
    search_keys = {
        "date":SearchKey.DATE
    },
    cv = CVType.time_series
)

In [36]:
enricher.fit(X_train, y_train,
             eval_set=[(X_test, y_test)])

[============================================================] 100% Finished


Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history

Detected task type: ModelTaskType.REGRESSION






Column name,Status,Errors
date,All valid,-
target,All valid,-



Running search request, search_id=fe25c81a-4687-4b3e-9ad6-67c4ccea82e5
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com


63 relevant feature(s) found with the search keys: ['date']


f_weather_date_weather_pca_0_d7e0a1fc,0.1586,100.0000,"30.267, 16.4704, -11.4679",Upgini,Weather & climate normals data,Daily
f_autofe_div_3e22df83,0.0192,100.0000,"0.0096, 0.01, -0.0085",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_59463574,0.0175,100.0000,"0.0656, 0.0581, 0.0628",Upgini,AutoFE: features from Markets data,Daily
f_autofe_div_fa0777db,0.0134,100.0000,"0.0019, 0.0017, -0.0006",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_20a12630,0.0093,100.0000,"-0.0046, 0.0164, -0.0184",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_mul_0cc09d2d,0.0084,100.0000,"-7.2161, -26.1265, 1.9892",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_e944536b,0.0081,100.0000,"0.8708, 1.1941, 0.9124",Upgini,AutoFE: features from Markets data,Daily
f_autofe_div_c217fef5,0.0074,85.6444,"0.2282, 0.7975, 2.0765",Upgini,AutoFE: features from Calendar data,Daily
f_autofe_mul_8916215e,0.0062,100.0000,"316.7944, 298.0013, 347.5301",Upgini,AutoFE: features from Markets data,Daily
f_autofe_div_cfd8d471,0.0050,100.0000,"1648.1158, 1110.6621, 1445.401...",Upgini,AutoFE: features from Markets data,Daily
f_autofe_mul_8a925382,0.0048,100.0000,"220.5272, 271.7544, 344.4198",Upgini,AutoFE: features from Markets data,Daily


Upgini,Weather & climate normals data,0.1603,10
Upgini,"AutoFE: features from Calendar data,Markets data",0.0804,23
Upgini,AutoFE: features from Markets data,0.0506,13
Upgini,AutoFE: features from Calendar data,0.0088,4
Upgini,World economic indicators,0.0008,4
Upgini,Markets data,0.0007,7
Upgini,Calendar data,0.0002,2


"Calendar data,Markets data",f_autofe_div_3e22df83,f_events_date_week_sin1_847b5db1,f_financial_date_crude_oil_1f195998,/
Markets data,f_autofe_div_59463574,f_financial_date_gold_7d_to_7d_1y_shift_a8127a43,f_financial_date_silver_e4e33014,/
"Calendar data,Markets data",f_autofe_div_fa0777db,f_events_date_week_cos2_b0a07cfc,f_financial_date_stoxx_043cbcd4,/
"Calendar data,Markets data",f_autofe_div_20a12630,f_events_date_year_cos1_9014a856,f_financial_date_crude_oil_1f195998,/
"Calendar data,Markets data",f_autofe_mul_0cc09d2d,f_events_date_year_cos1_9014a856,f_financial_date_crude_oil_1f195998,*
Markets data,f_autofe_div_e944536b,f_financial_date_gold_7d_to_1y_ae310379,f_financial_date_snp500_7d_to_7d_1y_shift_fa788602,/
Calendar data,f_autofe_div_c217fef5,f_events_date_week_cos3_7525fe31,f_events_date_week_sin3_0723e6e6,/
Markets data,f_autofe_mul_8916215e,f_financial_date_silver_7d_to_1y_0ccfe462,f_financial_date_stoxx_043cbcd4,*
Markets data,f_autofe_div_cfd8d471,f_financial_date_gold_bf71e733,f_financial_date_silver_7d_to_1y_0ccfe462,/
Markets data,f_autofe_mul_8a925382,f_financial_date_gold_7d_to_1y_ae310379,f_financial_date_stoxx_043cbcd4,*
"Calendar data,Markets data",f_autofe_div_b4a7249a,f_events_date_week_cos3_7525fe31,f_financial_date_silver_7d_to_1y_0ccfe462,/



Examples of outliers with maximum value of target:
8    179.0
6    177.0
5    174.0
Name: target, dtype: float64
Outliers will be excluded during the metrics calculation.
Calculating accuracy uplift after enrichment...


Train,9000,52.9562,257.3420,77.9357,179.4062
Eval 1,3000,52.1967,259.5413,77.6264,181.9149


In [37]:
model = CatBoostRegressor(verbose=False, allow_writing_files=False, random_state=0)

In [38]:
enricher.calculate_metrics(
    X_train, y_train,
    eval_set = [(X_test, y_test)],
    estimator = model,
    scoring = "mean_absolute_percentage_error"
)

Calculating accuracy uplift after enrichment...


,Dataset type,Rows,Mean target,Baseline mean_absolute_percentage_error,Enriched mean_absolute_percentage_error,Uplift
0,Train,9000,52.9562,0.277531,0.153019,0.124512
1,Eval 1,3000,52.1967,0.277770,0.154539,0.123231


In [39]:
enriched_X_train = enricher.transform(X_train, keep_input=True)
enriched_X_test = enricher.transform(X_test, keep_input=True)

[============================================================] 100% Finished

You use Trial access to Upgini data enrichment. Limit for Trial: 10000 rows. You have already enriched: 0 rows.

Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history




Column name,Status,Errors
date,All valid,-



Running search request, search_id=88ba26e3-2441-4f1c-b610-82e0067875b1
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com

Retrieving selected features from data sources...


[============================================================] 100% Finished

You use Trial access to Upgini data enrichment. Limit for Trial: 10000 rows. You have already enriched: 0 rows.

Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history




Column name,Status,Errors
date,All valid,-



Running search request, search_id=c06b9796-463d-448f-b004-e6504a33333a
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com

Retrieving selected features from data sources...


In [40]:
enriched_X_train.head()

,level_0,date,store,item,f_weather_date_weather_pca_0_d7e0a1fc,f_autofe_div_3e22df83,f_autofe_div_59463574,f_autofe_div_fa0777db,f_autofe_div_20a12630,f_autofe_mul_0cc09d2d,...,f_financial_date_natural_gas_92dac942,f_financial_date_silver_e4e33014,f_financial_date_snp500_gap_d0db897e,f_financial_date_stoxx_1d_to_7d_c615714e,f_weather_date_weather_pca_3_006924ed,f_weather_date_weather_umap_10_6b2de688,f_weather_date_weather_umap_11_c0a11c2b,f_weather_date_weather_umap_22_0342ee9e,f_weather_date_weather_umap_31_fa6d9a99,f_weather_date_weather_umap_48_b39cd0c4
5471,6826,2015-04-10,10,11,6.052820,-0.008543,0.057262,0.001524,-0.005923,-15.278638,...,2.528,16.164000,-0.609863,1.019471,-1.112664,4.666216,4.712911,5.340097,5.162955,5.184923
1890,2367,2013-10-16,10,7,-6.662315,0.009633,0.034423,-0.002862,0.004006,41.033138,...,3.790,21.146000,-0.969971,1.012148,-1.422222,2.976508,4.434123,4.951547,5.092441,5.760431
1684,2115,2013-09-17,2,9,-18.506748,0.007335,0.034517,-0.000710,-0.000766,-8.705887,...,3.738,21.962999,3.709961,1.006755,-5.242169,4.204010,4.396882,4.773106,4.837272,5.847764
11645,14548,2017-11-06,1,3,5.011266,0.000000,0.058497,0.002525,0.012626,39.088628,...,2.984,16.794001,0.000000,1.001228,1.384160,2.986120,4.040318,5.214069,5.085908,5.286087
10369,12944,2017-04-20,1,1,2.693706,0.008602,0.057358,0.001653,-0.009114,-23.188919,...,3.185,18.136000,4.600098,0.994049,0.203082,4.659188,5.536651,5.513693,5.139919,5.147007


Training our model with both, basic feature and enriched feature.

In [41]:
# Basic Features
model.fit(X_train, y_train)

In [42]:
y_pred = model.predict(X_test)

In [44]:
eval_metric(y_test.values, y_pred, "SMAPE")

[17.71411433973321]

In [45]:
# Using Enriched Features
model.fit(enriched_X_train, y_train)
y_enriched_pred = model.predict(enriched_X_test)
eval_metric(y_test.values, y_enriched_pred, "SMAPE")

[14.172598207592278]